# Session 15 - Data Cleaning : Nettoyage du Dataset Titanic

## 🎯 Objectifs
- Diagnostiquer les problèmes de qualité des données
- Traiter les valeurs manquantes avec différentes stratégies
- Créer de nouvelles variables pertinentes (feature engineering)
- Exporter un dataset propre et prêt pour l'analyse

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')

print("✓ Bibliothèques importées")

## Partie 1 : Chargement et diagnostic initial

In [ ]:
# Charger les données
df = pd.read_csv('../data/titanic.csv')

print(f"Dataset chargé : {df.shape[0]} lignes x {df.shape[1]} colonnes")
df.head()

In [ ]:
# Informations générales
print("=== INFORMATIONS SUR LE DATASET ===")
df.info()

In [ ]:
# Statistiques descriptives
print("=== STATISTIQUES DESCRIPTIVES ===")
df.describe()

### 1.1 : Analyse des valeurs manquantes

In [ ]:
# Compter les valeurs manquantes
missing_count = df.isnull().sum()
missing_pct = (missing_count / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    'Nombre': missing_count,
    'Pourcentage': missing_pct
})

print("=== VALEURS MANQUANTES ===")
print(missing_df[missing_df['Nombre'] > 0].sort_values('Nombre', ascending=False))

# Interprétation
print("\n📊 Analyse :")
print("- Cabin : 77% manquant → colonne peu exploitable")
print("- Age : 20% manquant → nécessite imputation")
print("- Embarked : 0.22% manquant → facile à imputer")

In [ ]:
# Visualiser les patterns de valeurs manquantes
plt.figure(figsize=(12, 6))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis', yticklabels=False)
plt.title('Patterns de valeurs manquantes (jaune = manquant)', fontsize=14)
plt.tight_layout()
plt.show()

### 1.2 : Détection des doublons

In [ ]:
# Vérifier les doublons complets
duplicates = df.duplicated()
print(f"Nombre de doublons complets : {duplicates.sum()}")

# Vérifier les doublons sur colonnes importantes
duplicates_name = df.duplicated(subset=['Name'], keep=False)
print(f"Passagers avec noms identiques : {duplicates_name.sum()}")

if duplicates_name.sum() > 0:
    print("\nExemples :")
    print(df[duplicates_name][['Name', 'Age', 'Pclass']].head(10))

### 1.3 : Détection des outliers

In [ ]:
# Analyser les outliers sur Age et Fare
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Age - Boxplot
axes[0, 0].boxplot(df['Age'].dropna())
axes[0, 0].set_title('Age - Boxplot')
axes[0, 0].set_ylabel('Âge')

# Age - Histogramme
axes[0, 1].hist(df['Age'].dropna(), bins=30, edgecolor='black')
axes[0, 1].set_title('Age - Distribution')
axes[0, 1].set_xlabel('Âge')
axes[0, 1].set_ylabel('Fréquence')

# Fare - Boxplot
axes[1, 0].boxplot(df['Fare'].dropna())
axes[1, 0].set_title('Fare - Boxplot')
axes[1, 0].set_ylabel('Prix')

# Fare - Histogramme
axes[1, 1].hist(df['Fare'].dropna(), bins=50, edgecolor='black')
axes[1, 1].set_title('Fare - Distribution')
axes[1, 1].set_xlabel('Prix')
axes[1, 1].set_ylabel('Fréquence')

plt.tight_layout()
plt.show()

In [ ]:
# Détection des outliers avec méthode IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Outliers sur Fare
fare_outliers, fare_lower, fare_upper = detect_outliers_iqr(df, 'Fare')
print(f"Outliers sur Fare : {len(fare_outliers)} ({len(fare_outliers)/len(df)*100:.1f}%)")
print(f"Bornes : [{fare_lower:.2f}, {fare_upper:.2f}]")
print(f"\nExemples de prix extrêmes :")
print(fare_outliers.nlargest(5, 'Fare')[['Name', 'Pclass', 'Fare']])

print("\n💡 Décision : Garder les outliers de Fare car ils correspondent à des cabines luxueuses (1ère classe)")

## Partie 2 : Nettoyage des données

In [ ]:
# Créer une copie pour le nettoyage
df_clean = df.copy()
print("✓ Copie créée pour le nettoyage")

### 2.1 : Traitement de la colonne Cabin

In [ ]:
# Cabin : 77% manquant → créer un indicateur puis supprimer
df_clean['HasCabin'] = df_clean['Cabin'].notna().astype(int)

print(f"Passagers avec cabine connue : {df_clean['HasCabin'].sum()} ({df_clean['HasCabin'].mean()*100:.1f}%)")
print(f"Taux de survie avec cabine : {df_clean[df_clean['HasCabin']==1]['Survived'].mean():.2%}")
print(f"Taux de survie sans cabine : {df_clean[df_clean['HasCabin']==0]['Survived'].mean():.2%}")

# Supprimer la colonne Cabin
df_clean = df_clean.drop('Cabin', axis=1)
print("\n✓ Colonne 'Cabin' supprimée, indicateur 'HasCabin' créé")

### 2.2 : Traitement de la colonne Age

In [ ]:
# Age : Imputation par médiane selon Pclass et Sex
print("=== IMPUTATION DE L'ÂGE ===")
print("Stratégie : Médiane par groupe (Pclass + Sex)\n")

# Voir les médianes par groupe
age_by_group = df_clean.groupby(['Pclass', 'Sex'])['Age'].median()
print("Âges médians par groupe :")
print(age_by_group)

# Créer un indicateur de valeur manquante
df_clean['Age_was_missing'] = df_clean['Age'].isnull().astype(int)

# Imputer
df_clean['Age'] = df_clean.groupby(['Pclass', 'Sex'])['Age'].transform(
    lambda x: x.fillna(x.median())
)

print(f"\n✓ {df_clean['Age_was_missing'].sum()} valeurs d'âge imputées")
print(f"✓ Aucune valeur manquante restante : {df_clean['Age'].isnull().sum() == 0}")

### 2.3 : Traitement de la colonne Embarked

In [ ]:
# Embarked : 2 valeurs manquantes → mode
print("=== IMPUTATION DU PORT D'EMBARQUEMENT ===")
print(f"Valeurs manquantes : {df_clean['Embarked'].isnull().sum()}")

# Voir la distribution
print("\nDistribution :")
print(df_clean['Embarked'].value_counts())

# Imputer avec le mode (valeur la plus fréquente)
mode_embarked = df_clean['Embarked'].mode()[0]
df_clean['Embarked'].fillna(mode_embarked, inplace=True)

print(f"\n✓ Valeurs manquantes imputées avec '{mode_embarked}' (Southampton)")
print(f"✓ Aucune valeur manquante restante : {df_clean['Embarked'].isnull().sum() == 0}")

### 2.4 : Traitement de la colonne Fare

In [ ]:
# Fare : Vérifier s'il y a des valeurs manquantes
print(f"Valeurs manquantes dans Fare : {df_clean['Fare'].isnull().sum()}")

if df_clean['Fare'].isnull().sum() > 0:
    # Imputer avec la médiane de la classe
    df_clean['Fare'] = df_clean.groupby('Pclass')['Fare'].transform(
        lambda x: x.fillna(x.median())
    )
    print("✓ Valeurs manquantes imputées")
else:
    print("✓ Aucune valeur manquante")

### 2.5 : Vérification finale des valeurs manquantes

In [ ]:
# Vérifier qu'il n'y a plus de valeurs manquantes (sauf colonnes à supprimer)
print("=== VÉRIFICATION FINALE ===")
missing_final = df_clean.isnull().sum()
print(missing_final[missing_final > 0])

if missing_final.sum() == 0:
    print("\n✅ Aucune valeur manquante ! Dataset prêt pour le feature engineering.")
else:
    print("\n⚠️ Il reste des valeurs manquantes à traiter")

## Partie 3 : Feature Engineering

### 3.1 : Taille de la famille

In [ ]:
# Créer FamilySize
df_clean['FamilySize'] = df_clean['SibSp'] + df_clean['Parch'] + 1

print("=== FAMILY SIZE ===")
print(f"Distribution :")
print(df_clean['FamilySize'].value_counts().sort_index())

print(f"\nTaille moyenne : {df_clean['FamilySize'].mean():.2f}")
print(f"Passagers seuls : {(df_clean['FamilySize']==1).sum()} ({(df_clean['FamilySize']==1).mean()*100:.1f}%)")

# Analyser la survie par taille de famille
print("\nTaux de survie par taille de famille :")
survival_by_family = df_clean.groupby('FamilySize')['Survived'].mean().sort_values(ascending=False)
for size, rate in survival_by_family.items():
    print(f"  Taille {size} : {rate:.2%}")

In [ ]:
# Créer IsAlone
df_clean['IsAlone'] = (df_clean['FamilySize'] == 1).astype(int)

print("Taux de survie :")
print(f"  Seuls (IsAlone=1) : {df_clean[df_clean['IsAlone']==1]['Survived'].mean():.2%}")
print(f"  En famille (IsAlone=0) : {df_clean[df_clean['IsAlone']==0]['Survived'].mean():.2%}")

print("\n✓ Variables 'FamilySize' et 'IsAlone' créées")

### 3.2 : Catégories de taille de famille

In [ ]:
# Catégoriser la taille de famille
def categorize_family(size):
    if size == 1:
        return 'Alone'
    elif size <= 3:
        return 'Small'
    elif size <= 5:
        return 'Medium'
    else:
        return 'Large'

df_clean['FamilyCategory'] = df_clean['FamilySize'].apply(categorize_family)

print("Distribution des catégories :")
print(df_clean['FamilyCategory'].value_counts())

print("\nTaux de survie par catégorie :")
print(df_clean.groupby('FamilyCategory')['Survived'].mean().sort_values(ascending=False))

### 3.3 : Extraction du titre

In [ ]:
# Extraire le titre du nom
df_clean['Title'] = df_clean['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

print("=== TITRES EXTRAITS ===")
print("Distribution des titres :")
print(df_clean['Title'].value_counts())

In [ ]:
# Simplifier les titres rares
title_mapping = {
    'Mr': 'Mr',
    'Miss': 'Miss',
    'Mrs': 'Mrs',
    'Master': 'Master',
    'Dr': 'Rare',
    'Rev': 'Rare',
    'Col': 'Rare',
    'Major': 'Rare',
    'Mlle': 'Miss',
    'Mme': 'Mrs',
    'Ms': 'Miss',
    'Lady': 'Rare',
    'Countess': 'Rare',
    'Capt': 'Rare',
    'Jonkheer': 'Rare',
    'Don': 'Rare',
    'Dona': 'Rare',
    'Sir': 'Rare'
}

df_clean['Title'] = df_clean['Title'].map(title_mapping)
df_clean['Title'].fillna('Rare', inplace=True)  # Autres titres → Rare

print("Distribution après simplification :")
print(df_clean['Title'].value_counts())

print("\nTaux de survie par titre :")
print(df_clean.groupby('Title')['Survived'].mean().sort_values(ascending=False))

print("\n✓ Variable 'Title' créée et simplifiée")

### 3.4 : Catégories d'âge

In [ ]:
# Créer des catégories d'âge
bins = [0, 12, 18, 35, 60, 100]
labels = ['Child', 'Teen', 'Young Adult', 'Adult', 'Senior']
df_clean['AgeGroup'] = pd.cut(df_clean['Age'], bins=bins, labels=labels)

print("=== GROUPES D'ÂGE ===")
print("Distribution :")
print(df_clean['AgeGroup'].value_counts().sort_index())

print("\nTaux de survie par groupe d'âge :")
print(df_clean.groupby('AgeGroup')['Survived'].mean())

print("\n✓ Variable 'AgeGroup' créée")

### 3.5 : Catégories de prix

In [ ]:
# Créer des catégories de prix (quartiles)
df_clean['FareCategory'] = pd.qcut(df_clean['Fare'], q=4, 
                                     labels=['Low', 'Medium', 'High', 'VeryHigh'])

print("=== CATÉGORIES DE PRIX ===")
print("Distribution :")
print(df_clean['FareCategory'].value_counts())

print("\nTaux de survie par catégorie de prix :")
print(df_clean.groupby('FareCategory')['Survived'].mean())

print("\n✓ Variable 'FareCategory' créée")

### 3.6 : Variables binaires supplémentaires

In [ ]:
# Femme ou enfant (priorité d'évacuation)
df_clean['WomanOrChild'] = ((df_clean['Sex'] == 'female') | (df_clean['Age'] < 18)).astype(int)

print("Taux de survie :")
print(f"  Femmes/Enfants : {df_clean[df_clean['WomanOrChild']==1]['Survived'].mean():.2%}")
print(f"  Hommes adultes : {df_clean[df_clean['WomanOrChild']==0]['Survived'].mean():.2%}")

# Classe supérieure (1ère ou 2ème)
df_clean['UpperClass'] = (df_clean['Pclass'] <= 2).astype(int)

print("\nTaux de survie :")
print(f"  Classes sup. (1&2) : {df_clean[df_clean['UpperClass']==1]['Survived'].mean():.2%}")
print(f"  3ème classe : {df_clean[df_clean['UpperClass']==0]['Survived'].mean():.2%}")

print("\n✓ Variables 'WomanOrChild' et 'UpperClass' créées")

## Partie 4 : Préparation finale et export

### 4.1 : Suppression des colonnes inutiles

In [ ]:
# Colonnes à supprimer
columns_to_drop = ['PassengerId', 'Name', 'Ticket']

print(f"Colonnes à supprimer : {columns_to_drop}")
df_clean = df_clean.drop(columns=columns_to_drop)

print(f"\n✓ Colonnes supprimées")
print(f"Dimensions finales : {df_clean.shape[0]} lignes x {df_clean.shape[1]} colonnes")

### 4.2 : Récapitulatif du dataset nettoyé

In [ ]:
# Afficher les colonnes finales
print("=== COLONNES FINALES ===")
for i, col in enumerate(df_clean.columns, 1):
    print(f"{i:2d}. {col:20s} - {df_clean[col].dtype}")

In [ ]:
# Statistiques finales
print("=== STATISTIQUES FINALES ===")
df_clean.describe()

In [ ]:
# Aperçu du dataset nettoyé
print("=== APERÇU DU DATASET NETTOYÉ ===")
df_clean.head(10)

In [ ]:
# Vérification finale : aucune valeur manquante
print("=== VÉRIFICATION FINALE ===")
missing = df_clean.isnull().sum()
if missing.sum() == 0:
    print("✅ Aucune valeur manquante dans le dataset !")
else:
    print("⚠️ Valeurs manquantes restantes :")
    print(missing[missing > 0])

### 4.3 : Export du dataset nettoyé

In [ ]:
# Sauvegarder le dataset nettoyé
output_path = '../data/titanic_clean.csv'
df_clean.to_csv(output_path, index=False)

print(f"✅ Dataset nettoyé sauvegardé : {output_path}")
print(f"   {df_clean.shape[0]} lignes x {df_clean.shape[1]} colonnes")
print(f"   Taille du fichier : {pd.read_csv(output_path).memory_usage(deep=True).sum() / 1024:.1f} KB")

## 📊 Résumé du nettoyage

### Actions effectuées :

#### ✅ Traitement des valeurs manquantes
- **Cabin** (77% manquant) : Créé indicateur `HasCabin`, puis supprimé
- **Age** (20% manquant) : Imputé par médiane selon `Pclass` et `Sex`
- **Embarked** (2 valeurs) : Imputé avec le mode (Southampton)

#### ✅ Feature Engineering
- **FamilySize** : `SibSp + Parch + 1`
- **IsAlone** : Indicateur de voyage seul
- **FamilyCategory** : Catégories de taille de famille
- **Title** : Extraction et simplification des titres
- **AgeGroup** : Catégories d'âge
- **FareCategory** : Catégories de prix (quartiles)
- **WomanOrChild** : Indicateur femme ou enfant
- **UpperClass** : Indicateur classe supérieure
- **Age_was_missing** : Indicateur d'imputation d'âge

#### ✅ Nettoyage
- Suppression des colonnes inutiles : `PassengerId`, `Name`, `Ticket`, `Cabin`
- Aucune valeur manquante restante
- Dataset prêt pour l'EDA et la modélisation

### Prochaines étapes (Session 16) :
- Exploration visuelle approfondie (EDA)
- Analyse des corrélations
- Réponse à des questions métier complexes